# Silver layer


Silver = staging curado e incremental: padroniza tipos, normaliza valores (status, datas), deduplica e prepara SCD. Tabelas "técnicas" com sufixos _clean e a SCD T2 operacional de equipamentos (ex.: `silver.equipment_scd`).


In [ ]:
CREATE SCHEMA IF NOT EXISTS `manufatura_lakehouse`.`silver`;
USE CATALOG `manufatura_lakehouse`;


## Abordagem incremental (Databricks / Delta)
Use `MERGE INTO` + watermark por coluna de tempo do evento/atualização (`last_update_date`, `reading_timestamp`, `last_update` ou `_ingestion_timestamp` se tiver). A Silver inteira pode (e deve) ser incremental:


### 1) Silver Equipment – limpeza incremental
Objetivo: tipar datas, padronizar status, remover nulos críticos e suportar duplicatas por `equipment_id`.
No primeiro build, cria a tabela; nos próximos, faça MERGE por NK (`equipment_id`).


In [ ]:
--DROP TABLE silver.equipment_clean;


In [ ]:
-- =========================================================
-- SILVER: equipment_clean (dedupe + idempotência via hash)
-- =========================================================
CREATE TABLE IF NOT EXISTS silver.equipment_clean (
  equipment_id      STRING,
  equipment_name    STRING,
  equipment_type    STRING,
  location          STRING,
  installation_date DATE,
  manufacturer      STRING,
  model             STRING,
  status            STRING,
  last_update_date  TIMESTAMP,
  row_hash          STRING
) USING DELTA;

-- 1) Stage: normaliza e parseia installation_date e last_update_date
CREATE OR REPLACE TEMP VIEW stage_equipment AS
SELECT
  equipment_id,
  equipment_name,
  equipment_type,
  location,
  -- Parse installation_date (pode ter hora, então tenta timestamp primeiro)
  CAST(COALESCE(
    try_to_timestamp(installation_date, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(installation_date, "yyyy-MM-dd'T'HH:mm:ss"),
    try_to_timestamp(installation_date, 'dd-MM-yyyy HH:mm:ss'),
    try_to_timestamp(installation_date, 'yyyy-MM-dd'),
    try_to_timestamp(installation_date, 'yyyy/MM/dd'),
    try_to_timestamp(installation_date, 'dd/MM/yyyy'),
    try_to_timestamp(installation_date, 'dd-MM-yyyy'),
    try_to_date(installation_date, 'yyyy-MM-dd'),
    try_to_date(installation_date, 'yyyy/MM/dd'),
    try_to_date(installation_date, 'dd/MM/yyyy'),
    try_to_date(installation_date, 'dd-MM-yyyy')
  ) AS DATE) AS installation_date_parsed,
  manufacturer,
  model,
  UPPER(TRIM(status)) AS status_norm,
  -- Parse last_update_date para timestamp
  COALESCE(
    try_to_timestamp(last_update_date, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(last_update_date, "yyyy-MM-dd'T'HH:mm:ss"),
    try_to_timestamp(last_update_date, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(last_update_date, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(last_update_date, 'dd-MM-yyyy HH:mm:ss'),
    try_to_timestamp(last_update_date, 'yyyy-MM-dd'),
    try_to_timestamp(last_update_date, 'yyyy/MM/dd'),
    try_to_timestamp(last_update_date, 'dd/MM/yyyy'),
    try_to_timestamp(last_update_date, 'dd-MM-yyyy')
  ) AS last_update_ts
FROM bronze.equipment_master
WHERE equipment_id IS NOT NULL;

-- 2) Janela incremental (watermark de 90 dias)
-- Inclui registros com last_update_ts NULL para garantir que não perdemos dados
-- 
-- ⚠️ PARA FULL LOAD (primeira execução): Comente a linha WHERE abaixo e use:
--    CREATE OR REPLACE TEMP VIEW stage_equipment_win AS
--    SELECT * FROM stage_equipment;
--
CREATE OR REPLACE TEMP VIEW stage_equipment_win AS
SELECT *
FROM stage_equipment
WHERE last_update_ts IS NULL 
   OR last_update_ts >= date_sub(current_timestamp(), 90);

-- 3) Dedup: mantém 1 linha por equipment_id (mais recente por last_update_ts)
CREATE OR REPLACE TEMP VIEW stage_equipment_dedup AS
SELECT
  equipment_id,
  equipment_name,
  equipment_type,
  location,
  installation_date_parsed AS installation_date,
  manufacturer,
  model,
  status_norm AS status,
  last_update_ts AS last_update_date
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY equipment_id
      ORDER BY last_update_ts DESC NULLS LAST,
               equipment_name DESC
    ) AS rn
  FROM stage_equipment_win s
) z
WHERE rn = 1;

-- 4) Calcula hash para idempotência
CREATE OR REPLACE TEMP VIEW stage_equipment_final AS
SELECT
  equipment_id,
  equipment_name,
  equipment_type,
  location,
  installation_date,
  manufacturer,
  model,
  status,
  last_update_date,
  sha2(concat_ws('||',
    coalesce(equipment_name,''),
    coalesce(equipment_type,''),
    coalesce(location,''),
    coalesce(date_format(installation_date,'yyyy-MM-dd'),''),
    coalesce(manufacturer,''),
    coalesce(model,''),
    coalesce(status,'')
  ), 256) AS row_hash
FROM stage_equipment_dedup;

-- 5) MERGE idempotente
MERGE INTO silver.equipment_clean AS t
USING stage_equipment_final AS s
ON t.equipment_id = s.equipment_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.equipment_name   = s.equipment_name,
  t.equipment_type   = s.equipment_type,
  t.location         = s.location,
  t.installation_date = s.installation_date,
  t.manufacturer     = s.manufacturer,
  t.model            = s.model,
  t.status           = s.status,
  t.last_update_date = s.last_update_date,
  t.row_hash         = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  equipment_id, equipment_name, equipment_type, location, installation_date,
  manufacturer, model, status, last_update_date, row_hash
) VALUES (
  s.equipment_id, s.equipment_name, s.equipment_type, s.location, s.installation_date,
  s.manufacturer, s.model, s.status, s.last_update_date, s.row_hash
);


### 2) Silver Equipment – SCD Type 2 incremental
Mantenha a tabela técnica SCD na Silver (com `effective_start`, `effective_end`, `is_current`) e gere a dim de negócio na Gold. O material já dá um esqueleto de SCD2 com `MERGE` e current flag.


In [ ]:
--DROP TABLE silver.equipment_scd;


In [ ]:
-- ============================================================
-- SCD TYPE 2 - Equipment (Silver) com HASH + DEDUP incremental
-- ============================================================
CREATE TABLE IF NOT EXISTS silver.equipment_scd (
  equipment_sk      BIGINT GENERATED ALWAYS AS IDENTITY,
  equipment_id      STRING,
  equipment_name    STRING,
  equipment_type    STRING,
  location          STRING,
  manufacturer      STRING,
  model             STRING,
  status            STRING,
  effective_start   TIMESTAMP,
  effective_end     TIMESTAMP,
  is_current        BOOLEAN,
  row_hash          STRING
) USING DELTA;

-- Backfill do hash se necessário
UPDATE silver.equipment_scd
SET row_hash = COALESCE(row_hash,
  sha2(concat_ws('||',
    coalesce(equipment_name,''),
    coalesce(equipment_type,''),
    coalesce(location,''),
    coalesce(manufacturer,''),
    coalesce(model,''),
    coalesce(status,'')
  ), 256)
);

-- Stage bruto: parse robusto do last_update_date -> src_ts
CREATE OR REPLACE TEMP VIEW stage_equipment_scd_raw AS
SELECT
  equipment_id,
  equipment_name,
  equipment_type,
  location,
  manufacturer,
  model,
  UPPER(TRIM(status)) AS status_norm,
  COALESCE(
    try_to_timestamp(last_update_date, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(last_update_date, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(last_update_date, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(last_update_date, 'dd-MM-yyyy HH:mm:ss'),
    try_to_timestamp(last_update_date, 'yyyy-MM-dd'),
    try_to_timestamp(last_update_date, 'yyyy/MM/dd'),
    try_to_timestamp(last_update_date, 'dd/MM/yyyy'),
    try_to_timestamp(last_update_date, 'dd-MM-yyyy')
  ) AS src_ts
FROM bronze.equipment_master
WHERE equipment_id IS NOT NULL;

-- Janela incremental (watermark de 90 dias)
CREATE OR REPLACE TEMP VIEW stage_equipment_scd_window AS
SELECT *
FROM stage_equipment_scd_raw
WHERE COALESCE(src_ts, current_timestamp()) >= date_sub(current_timestamp(), 90);

-- Dedup por equipment_id (última versão por src_ts)
CREATE OR REPLACE TEMP VIEW stage_equipment_scd_latest AS
SELECT
  equipment_id,
  equipment_name,
  equipment_type,
  location,
  manufacturer,
  model,
  status_norm AS status,
  src_ts
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY equipment_id
      ORDER BY src_ts DESC NULLS LAST,
               equipment_name DESC,
               location DESC,
               status_norm DESC
    ) AS rn
  FROM stage_equipment_scd_window s
) z
WHERE rn = 1;

-- Calcula hash da "linha de negócio"
CREATE OR REPLACE TEMP VIEW stage_equipment_scd_hash AS
SELECT
  equipment_id,
  equipment_name,
  equipment_type,
  location,
  manufacturer,
  model,
  status,
  src_ts,
  sha2(concat_ws('||',
    coalesce(equipment_name,''),
    coalesce(equipment_type,''),
    coalesce(location,''),
    coalesce(manufacturer,''),
    coalesce(model,''),
    coalesce(status,'')
  ), 256) AS source_hash
FROM stage_equipment_scd_latest;

-- Expirar versões correntes QUE mudaram
MERGE INTO silver.equipment_scd AS tgt
USING stage_equipment_scd_hash AS src
ON  tgt.equipment_id = src.equipment_id
AND tgt.is_current  = TRUE
WHEN MATCHED AND tgt.row_hash <> src.source_hash THEN
  UPDATE SET
    tgt.effective_end = COALESCE(src.src_ts, current_timestamp()),
    tgt.is_current    = FALSE;

-- Inserir primeira versão OU nova versão apenas quando necessário
INSERT INTO silver.equipment_scd (
  equipment_id, equipment_name, equipment_type, location, manufacturer, model, status,
  effective_start, effective_end, is_current, row_hash
)
SELECT
  s.equipment_id,
  s.equipment_name,
  s.equipment_type,
  s.location,
  s.manufacturer,
  s.model,
  s.status,
  COALESCE(s.src_ts, current_timestamp()) AS effective_start,
  TIMESTAMP('9999-12-31')                 AS effective_end,
  TRUE                                    AS is_current,
  s.source_hash                           AS row_hash
FROM stage_equipment_scd_hash s
LEFT JOIN silver.equipment_scd c
  ON c.equipment_id = s.equipment_id AND c.is_current = TRUE
WHERE c.equipment_id IS NULL           -- novo equipamento
   OR c.row_hash <> s.source_hash;     -- mudança real


### 3) Silver IoT Readings – limpeza incremental
Objetivo: tipar valores numéricos, normalizar timestamps, deduplicar por (equipment_id, sensor_id, reading_timestamp).


In [ ]:
--DROP TABLE silver.iot_readings_clean;


In [ ]:
-- =========================================================
-- SILVER: iot_readings_clean (dedupe + idempotência via hash)
-- =========================================================
CREATE TABLE IF NOT EXISTS silver.iot_readings_clean (
  reading_id        STRING,
  equipment_id       STRING,
  sensor_id          STRING,
  sensor_type        STRING,
  reading_value      DECIMAL(18,4),
  reading_timestamp  TIMESTAMP,
  unit               STRING,
  row_hash           STRING
) USING DELTA;

-- 1) Stage: normaliza tipos e parseia reading_timestamp
CREATE OR REPLACE TEMP VIEW stage_iot_readings AS
SELECT
  reading_id,
  equipment_id,
  sensor_id,
  LOWER(TRIM(sensor_type)) AS sensor_type_norm,
  CAST(regexp_replace(reading_value, ',', '.') AS DECIMAL(18,4)) AS reading_value_norm,
  COALESCE(
    try_to_timestamp(reading_timestamp, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(reading_timestamp, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(reading_timestamp, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(reading_timestamp, 'dd-MM-yyyy HH:mm:ss'),
    try_to_timestamp(reading_timestamp, 'yyyy-MM-dd'),
    try_to_timestamp(reading_timestamp, 'yyyy/MM/dd'),
    try_to_timestamp(reading_timestamp, 'dd/MM/yyyy'),
    try_to_timestamp(reading_timestamp, 'dd-MM-yyyy')
  ) AS parsed_timestamp,
  unit
FROM bronze.iot_sensor_readings
WHERE equipment_id IS NOT NULL
  AND sensor_id IS NOT NULL;

-- 2) Janela incremental (watermark de 60 dias)
CREATE OR REPLACE TEMP VIEW stage_iot_readings_win AS
SELECT *
FROM stage_iot_readings
WHERE parsed_timestamp >= date_sub(current_timestamp(), 60);

-- 3) Dedup: mantém 1 linha por (equipment_id, sensor_id, reading_timestamp)
CREATE OR REPLACE TEMP VIEW stage_iot_readings_dedup AS
SELECT
  reading_id,
  equipment_id,
  sensor_id,
  sensor_type_norm AS sensor_type,
  reading_value_norm AS reading_value,
  parsed_timestamp AS reading_timestamp,
  unit
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY equipment_id, sensor_id, parsed_timestamp
      ORDER BY parsed_timestamp DESC NULLS LAST,
               reading_id DESC
    ) AS rn
  FROM stage_iot_readings_win s
  WHERE parsed_timestamp IS NOT NULL
) z
WHERE rn = 1;

-- 4) Calcula hash da linha
CREATE OR REPLACE TEMP VIEW stage_iot_readings_final AS
SELECT
  reading_id,
  equipment_id,
  sensor_id,
  sensor_type,
  reading_value,
  reading_timestamp,
  unit,
  sha2(concat_ws('||',
    equipment_id,
    sensor_id,
    sensor_type,
    cast(coalesce(reading_value,0) as string),
    coalesce(date_format(reading_timestamp,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(unit,'')
  ), 256) AS row_hash
FROM stage_iot_readings_dedup;

-- 5) MERGE idempotente
MERGE INTO silver.iot_readings_clean AS t
USING stage_iot_readings_final AS s
ON  t.reading_id = s.reading_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.equipment_id      = s.equipment_id,
  t.sensor_id         = s.sensor_id,
  t.sensor_type       = s.sensor_type,
  t.reading_value     = s.reading_value,
  t.reading_timestamp = s.reading_timestamp,
  t.unit              = s.unit,
  t.row_hash          = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  reading_id, equipment_id, sensor_id, sensor_type, reading_value, reading_timestamp, unit, row_hash
) VALUES (
  s.reading_id, s.equipment_id, s.sensor_id, s.sensor_type, s.reading_value, s.reading_timestamp, s.unit, s.row_hash
);


### 4) Silver Production Orders – limpeza incremental
Objetivo: tipar datas e quantidades, normalizar status, deduplicar por production_order_id.


In [ ]:
--DROP TABLE silver.production_orders_clean;


In [ ]:
-- =========================================================
-- SILVER: production_orders_clean (dedupe + idempotência via hash)
-- =========================================================
CREATE TABLE IF NOT EXISTS silver.production_orders_clean (
  production_order_id STRING,
  equipment_id        STRING,
  product_id          STRING,
  planned_start       TIMESTAMP,
  planned_end         TIMESTAMP,
  actual_start        TIMESTAMP,
  actual_end          TIMESTAMP,
  planned_quantity    INT,
  actual_quantity     INT,
  status              STRING,
  last_update         TIMESTAMP,
  row_hash            STRING
) USING DELTA;

CREATE OR REPLACE TEMP VIEW stage_production_orders AS
SELECT
  production_order_id,
  equipment_id,
  product_id,
  COALESCE(
    try_to_timestamp(planned_start, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(planned_start, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(planned_start, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(planned_start, 'yyyy-MM-dd'),
    try_to_timestamp(planned_start, 'yyyy/MM/dd')
  ) AS planned_start_ts,
  COALESCE(
    try_to_timestamp(planned_end, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(planned_end, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(planned_end, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(planned_end, 'yyyy-MM-dd'),
    try_to_timestamp(planned_end, 'yyyy/MM/dd')
  ) AS planned_end_ts,
  COALESCE(
    try_to_timestamp(actual_start, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(actual_start, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(actual_start, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(actual_start, 'yyyy-MM-dd'),
    try_to_timestamp(actual_start, 'yyyy/MM/dd')
  ) AS actual_start_ts,
  COALESCE(
    try_to_timestamp(actual_end, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(actual_end, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(actual_end, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(actual_end, 'yyyy-MM-dd'),
    try_to_timestamp(actual_end, 'yyyy/MM/dd')
  ) AS actual_end_ts,
  CAST(regexp_replace(planned_quantity, ',', '.') AS INT) AS planned_quantity_norm,
  CAST(regexp_replace(actual_quantity, ',', '.') AS INT) AS actual_quantity_norm,
  UPPER(TRIM(status)) AS status_norm,
  COALESCE(
    try_to_timestamp(last_update, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(last_update, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy-MM-dd'),
    try_to_timestamp(last_update, 'yyyy/MM/dd')
  ) AS last_update_ts
FROM bronze.production_orders
WHERE production_order_id IS NOT NULL;

CREATE OR REPLACE TEMP VIEW stage_production_orders_win AS
SELECT *
FROM stage_production_orders
WHERE last_update_ts >= date_sub(current_timestamp(), 60);

CREATE OR REPLACE TEMP VIEW stage_production_orders_dedup AS
SELECT
  production_order_id,
  equipment_id,
  product_id,
  planned_start_ts AS planned_start,
  planned_end_ts AS planned_end,
  actual_start_ts AS actual_start,
  actual_end_ts AS actual_end,
  planned_quantity_norm AS planned_quantity,
  actual_quantity_norm AS actual_quantity,
  status_norm AS status,
  last_update_ts AS last_update
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY production_order_id
      ORDER BY last_update_ts DESC NULLS LAST,
               production_order_id DESC
    ) AS rn
  FROM stage_production_orders_win s
  WHERE last_update_ts IS NOT NULL
) z
WHERE rn = 1;

CREATE OR REPLACE TEMP VIEW stage_production_orders_final AS
SELECT
  production_order_id,
  equipment_id,
  product_id,
  planned_start,
  planned_end,
  actual_start,
  actual_end,
  planned_quantity,
  actual_quantity,
  status,
  last_update,
  sha2(concat_ws('||',
    coalesce(equipment_id,''),
    coalesce(product_id,''),
    coalesce(date_format(planned_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(planned_end,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(actual_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(actual_end,'yyyy-MM-dd HH:mm:ss'),''),
    cast(coalesce(planned_quantity,0) as string),
    cast(coalesce(actual_quantity,0) as string),
    coalesce(status,'')
  ), 256) AS row_hash
FROM stage_production_orders_dedup;

MERGE INTO silver.production_orders_clean AS t
USING stage_production_orders_final AS s
ON t.production_order_id = s.production_order_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.equipment_id     = s.equipment_id,
  t.product_id       = s.product_id,
  t.planned_start    = s.planned_start,
  t.planned_end      = s.planned_end,
  t.actual_start     = s.actual_start,
  t.actual_end       = s.actual_end,
  t.planned_quantity = s.planned_quantity,
  t.actual_quantity  = s.actual_quantity,
  t.status           = s.status,
  t.last_update      = s.last_update,
  t.row_hash         = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  production_order_id, equipment_id, product_id, planned_start, planned_end,
  actual_start, actual_end, planned_quantity, actual_quantity, status, last_update, row_hash
) VALUES (
  s.production_order_id, s.equipment_id, s.product_id, s.planned_start, s.planned_end,
  s.actual_start, s.actual_end, s.planned_quantity, s.actual_quantity, s.status, s.last_update, s.row_hash
);


### 5) Silver Maintenance Orders – limpeza incremental
Objetivo: tipar datas, normalizar tipos e status, deduplicar por maintenance_order_id.


In [ ]:
--DROP TABLE silver.maintenance_orders_clean;


In [ ]:
-- =========================================================
-- SILVER: maintenance_orders_clean (dedupe + idempotência via hash)
-- =========================================================
CREATE TABLE IF NOT EXISTS silver.maintenance_orders_clean (
  maintenance_order_id STRING,
  equipment_id         STRING,
  maintenance_type     STRING,
  scheduled_start      TIMESTAMP,
  scheduled_end        TIMESTAMP,
  actual_start         TIMESTAMP,
  actual_end           TIMESTAMP,
  technician_id        STRING,
  status               STRING,
  priority             STRING,
  description          STRING,
  last_update          TIMESTAMP,
  row_hash             STRING
) USING DELTA;

CREATE OR REPLACE TEMP VIEW stage_maintenance_orders AS
SELECT
  maintenance_order_id,
  equipment_id,
  LOWER(TRIM(maintenance_type)) AS maintenance_type_norm,
  COALESCE(
    try_to_timestamp(scheduled_start, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(scheduled_start, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(scheduled_start, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(scheduled_start, 'yyyy-MM-dd'),
    try_to_timestamp(scheduled_start, 'yyyy/MM/dd')
  ) AS scheduled_start_ts,
  COALESCE(
    try_to_timestamp(scheduled_end, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(scheduled_end, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(scheduled_end, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(scheduled_end, 'yyyy-MM-dd'),
    try_to_timestamp(scheduled_end, 'yyyy/MM/dd')
  ) AS scheduled_end_ts,
  COALESCE(
    try_to_timestamp(actual_start, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(actual_start, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(actual_start, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(actual_start, 'yyyy-MM-dd'),
    try_to_timestamp(actual_start, 'yyyy/MM/dd')
  ) AS actual_start_ts,
  COALESCE(
    try_to_timestamp(actual_end, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(actual_end, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(actual_end, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(actual_end, 'yyyy-MM-dd'),
    try_to_timestamp(actual_end, 'yyyy/MM/dd')
  ) AS actual_end_ts,
  technician_id,
  UPPER(TRIM(status)) AS status_norm,
  LOWER(TRIM(priority)) AS priority_norm,
  description,
  COALESCE(
    try_to_timestamp(last_update, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(last_update, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy-MM-dd'),
    try_to_timestamp(last_update, 'yyyy/MM/dd')
  ) AS last_update_ts
FROM bronze.maintenance_orders
WHERE maintenance_order_id IS NOT NULL;

CREATE OR REPLACE TEMP VIEW stage_maintenance_orders_win AS
SELECT *
FROM stage_maintenance_orders
WHERE last_update_ts >= date_sub(current_timestamp(), 60);

CREATE OR REPLACE TEMP VIEW stage_maintenance_orders_dedup AS
SELECT
  maintenance_order_id,
  equipment_id,
  maintenance_type_norm AS maintenance_type,
  scheduled_start_ts AS scheduled_start,
  scheduled_end_ts AS scheduled_end,
  actual_start_ts AS actual_start,
  actual_end_ts AS actual_end,
  technician_id,
  status_norm AS status,
  priority_norm AS priority,
  description,
  last_update_ts AS last_update
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY maintenance_order_id
      ORDER BY last_update_ts DESC NULLS LAST,
               maintenance_order_id DESC
    ) AS rn
  FROM stage_maintenance_orders_win s
  WHERE last_update_ts IS NOT NULL
) z
WHERE rn = 1;

CREATE OR REPLACE TEMP VIEW stage_maintenance_orders_final AS
SELECT
  maintenance_order_id,
  equipment_id,
  maintenance_type,
  scheduled_start,
  scheduled_end,
  actual_start,
  actual_end,
  technician_id,
  status,
  priority,
  description,
  last_update,
  sha2(concat_ws('||',
    coalesce(equipment_id,''),
    coalesce(maintenance_type,''),
    coalesce(date_format(scheduled_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(scheduled_end,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(actual_start,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(date_format(actual_end,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(technician_id,''),
    coalesce(status,''),
    coalesce(priority,'')
  ), 256) AS row_hash
FROM stage_maintenance_orders_dedup;

MERGE INTO silver.maintenance_orders_clean AS t
USING stage_maintenance_orders_final AS s
ON t.maintenance_order_id = s.maintenance_order_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.equipment_id     = s.equipment_id,
  t.maintenance_type = s.maintenance_type,
  t.scheduled_start  = s.scheduled_start,
  t.scheduled_end    = s.scheduled_end,
  t.actual_start     = s.actual_start,
  t.actual_end       = s.actual_end,
  t.technician_id    = s.technician_id,
  t.status          = s.status,
  t.priority         = s.priority,
  t.description      = s.description,
  t.last_update      = s.last_update,
  t.row_hash         = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  maintenance_order_id, equipment_id, maintenance_type, scheduled_start, scheduled_end,
  actual_start, actual_end, technician_id, status, priority, description, last_update, row_hash
) VALUES (
  s.maintenance_order_id, s.equipment_id, s.maintenance_type, s.scheduled_start, s.scheduled_end,
  s.actual_start, s.actual_end, s.technician_id, s.status, s.priority, s.description, s.last_update, s.row_hash
);


### 6) Silver Quality Inspections – limpeza incremental
Objetivo: tipar datas e valores, normalizar códigos de defeito, deduplicar por inspection_id.


In [ ]:
--DROP TABLE silver.quality_inspections_clean;


In [ ]:
-- =========================================================
-- SILVER: quality_inspections_clean (dedupe + idempotência via hash)
-- =========================================================
CREATE TABLE IF NOT EXISTS silver.quality_inspections_clean (
  inspection_id       STRING,
  production_order_id  STRING,
  equipment_id           STRING,
  inspection_type     STRING,
  inspection_date     TIMESTAMP,
  inspector_id        STRING,
  passed              BOOLEAN,
  failed_quantity     INT,
  total_quantity      INT,
  defect_codes        STRING,
  notes               STRING,
  last_update         TIMESTAMP,
  row_hash            STRING
) USING DELTA;

CREATE OR REPLACE TEMP VIEW stage_quality_inspections AS
SELECT
  inspection_id,
  production_order_id,
  equipment_id,
  LOWER(TRIM(inspection_type)) AS inspection_type_norm,
  COALESCE(
    try_to_timestamp(inspection_date, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(inspection_date, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(inspection_date, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(inspection_date, 'yyyy-MM-dd'),
    try_to_timestamp(inspection_date, 'yyyy/MM/dd')
  ) AS inspection_date_ts,
  inspector_id,
  CASE
    WHEN LOWER(TRIM(passed)) IN ('true', '1', 'yes', 'y', 't') THEN TRUE
    WHEN LOWER(TRIM(passed)) IN ('false', '0', 'no', 'n', 'f') THEN FALSE
    ELSE NULL
  END AS passed_bool,
  CAST(regexp_replace(failed_quantity, ',', '.') AS INT) AS failed_quantity_norm,
  CAST(regexp_replace(total_quantity, ',', '.') AS INT) AS total_quantity_norm,
  UPPER(TRIM(defect_codes)) AS defect_codes_norm,
  notes,
  COALESCE(
    try_to_timestamp(last_update, 'yyyy-MM-dd HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy/MM/dd HH:mm:ss'),
    try_to_timestamp(last_update, 'dd/MM/yyyy HH:mm:ss'),
    try_to_timestamp(last_update, 'yyyy-MM-dd'),
    try_to_timestamp(last_update, 'yyyy/MM/dd')
  ) AS last_update_ts
FROM bronze.quality_inspections
WHERE inspection_id IS NOT NULL;

CREATE OR REPLACE TEMP VIEW stage_quality_inspections_win AS
SELECT *
FROM stage_quality_inspections
WHERE last_update_ts >= date_sub(current_timestamp(), 60);

CREATE OR REPLACE TEMP VIEW stage_quality_inspections_dedup AS
SELECT
  inspection_id,
  production_order_id,
  equipment_id,
  inspection_type_norm AS inspection_type,
  inspection_date_ts AS inspection_date,
  inspector_id,
  passed_bool AS passed,
  failed_quantity_norm AS failed_quantity,
  total_quantity_norm AS total_quantity,
  defect_codes_norm AS defect_codes,
  notes,
  last_update_ts AS last_update
FROM (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY inspection_id
      ORDER BY last_update_ts DESC NULLS LAST,
               inspection_id DESC
    ) AS rn
  FROM stage_quality_inspections_win s
  WHERE last_update_ts IS NOT NULL
) z
WHERE rn = 1;

CREATE OR REPLACE TEMP VIEW stage_quality_inspections_final AS
SELECT
  inspection_id,
  production_order_id,
  equipment_id,
  inspection_type,
  inspection_date,
  inspector_id,
  passed,
  failed_quantity,
  total_quantity,
  defect_codes,
  notes,
  last_update,
  sha2(concat_ws('||',
    coalesce(production_order_id,''),
    coalesce(equipment_id,''),
    coalesce(inspection_type,''),
    coalesce(date_format(inspection_date,'yyyy-MM-dd HH:mm:ss'),''),
    coalesce(inspector_id,''),
    cast(coalesce(passed,false) as string),
    cast(coalesce(failed_quantity,0) as string),
    cast(coalesce(total_quantity,0) as string),
    coalesce(defect_codes,'')
  ), 256) AS row_hash
FROM stage_quality_inspections_dedup;

MERGE INTO silver.quality_inspections_clean AS t
USING stage_quality_inspections_final AS s
ON t.inspection_id = s.inspection_id
WHEN MATCHED AND (t.row_hash IS NULL OR t.row_hash <> s.row_hash) THEN UPDATE SET
  t.production_order_id = s.production_order_id,
  t.equipment_id        = s.equipment_id,
  t.inspection_type     = s.inspection_type,
  t.inspection_date     = s.inspection_date,
  t.inspector_id        = s.inspector_id,
  t.passed              = s.passed,
  t.failed_quantity     = s.failed_quantity,
  t.total_quantity      = s.total_quantity,
  t.defect_codes        = s.defect_codes,
  t.notes               = s.notes,
  t.last_update         = s.last_update,
  t.row_hash            = s.row_hash
WHEN NOT MATCHED THEN INSERT (
  inspection_id, production_order_id, equipment_id, inspection_type, inspection_date,
  inspector_id, passed, failed_quantity, total_quantity, defect_codes, notes, last_update, row_hash
) VALUES (
  s.inspection_id, s.production_order_id, s.equipment_id, s.inspection_type, s.inspection_date,
  s.inspector_id, s.passed, s.failed_quantity, s.total_quantity, s.defect_codes, s.notes, s.last_update, s.row_hash
);
